In [397]:
import os
import cv2 as cv
import numpy as np
import pickle
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

ruta_base = r'C:\Users\Royer\Downloads\onta_waldo\onta_waldo\Dataset_waldo'
positivas_dir = os.path.join(ruta_base, 'waldo')
negativas_dir = os.path.join(ruta_base, 'no_waldo')

def extract_features(img, size=(64, 128)):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    resized = cv.resize(gray, size)

    hog = cv.HOGDescriptor(_winSize=(size[0] // 8 * 8, size[1] // 8 * 8),
                           _blockSize=(16, 16),
                           _blockStride=(8, 8),
                           _cellSize=(8, 8),
                           _nbins=9)
    h = hog.compute(resized)
    return h.flatten()

data = []
labels = []

for img_file in os.listdir(positivas_dir):
    img_path = os.path.join(positivas_dir, img_file)
    img = cv.imread(img_path)
    if img is not None:
        features = extract_features(img)
        data.append(features)
        labels.append(1)

for img_file in os.listdir(negativas_dir):
    img_path = os.path.join(negativas_dir, img_file)
    img = cv.imread(img_path)
    if img is not None:
        features = extract_features(img)
        data.append(features)
        labels.append(0) 

data = np.array(data)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.5, random_state=80)

svm_model = svm.SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy * 100:.2f}%')

with open('wally_svm_model2.pkl', 'wb') as f:
    pickle.dump(svm_model, f)
print("Modelo SVM guardado como 'wally_svm_model2.pkl'")

Precisión del modelo: 85.49%
Modelo SVM guardado como 'wally_svm_model2.pkl'


In [422]:
import cv2 as cv
import numpy as np
import pickle
import os

with open('wally_svm_model2.pkl', 'rb') as f:
    svm_model = pickle.load(f)

def extract_features(img, size=(64, 128)):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    resized = cv.resize(gray, size)
    hog = cv.HOGDescriptor()
    h = hog.compute(resized)
    return h.flatten()

image_path = 'C:\\Users\\Royer\\Downloads\\onta_waldo\\onta_waldo\\ejemplos\\4.png'
image = cv.imread(image_path)

# Ajuste del tamaño de la ventana de detección
window_size = (138, 138)  # Puedes experimentar con diferentes tamaños de ventana

# Ajuste del paso de la ventana de detección
step_size = 63 # Puedes ajustar el tamaño del paso para aumentar la precisión

found = False
detections = []

for y in range(0, image.shape[0] - window_size[1], step_size):
    for x in range(0, image.shape[1] - window_size[0], step_size):
        sub_img = image[y:y + window_size[1], x:x + window_size[0]]
        features = extract_features(sub_img)
        prediction = svm_model.predict([features])
        if prediction == 1:
            detections.append((x, y, window_size[0], window_size[1]))

def non_max_suppression(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)
    pick = []

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 0] + boxes[:, 2]
    y2 = boxes[:, 1] + boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx1 - xx2 + 1)
        h = np.maximum(0, yy1 - yy2 + 1)

        overlap = (w * h) / area[idxs[:last]]

        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

# Ajuste del umbral de superposición para la supresión no máxima
detections = non_max_suppression(detections, overlapThresh=0.1)  # Puedes experimentar con este valor

for (x, y, w, h) in detections:
    cv.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv.putText(image, 'Waldo', (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    found = True

if not found:
    print("IDK.")

height, width, _ = image.shape

resize_factor = 0.6  
new_width = int(width * resize_factor)
new_height = int(height * resize_factor)

cv.namedWindow('Donde esta Waldo', cv.WINDOW_NORMAL)
cv.resizeWindow('Donde esta Waldo', new_width, new_height)
cv.imshow('Donde esta Waldo', image)
cv.waitKey(0)
cv.destroyAllWindows()
